How do you find the shared 15-mer regulatory motif in the below strings?
```
1 "atgaccgggatactgataaaaaaaagggggggggcgtacacattagataaacgtatgaagtacgttagactcggcgccgccg"
2 "acccctattttttgagcagatttagtgacctggaaaaaaaatttgagtacaaaacttttccgaataaaaaaaaaggggggga"
3 "tgagtatccctgggatgacttaaaaaaaagggggggtgctctcccgatttttgaatatgtaggatcattcgccagggtccga"
4 "gctgagaattggatgaaaaaaaagggggggtccacgcaatcgcgaaccaacgcggacccaaaggcaagaccgataaaggaga"
5 "tcccttttgcggtaatgtgccgggaggctggttacgtagggaagccctaacggacttaataaaaaaaagggggggcttatag"
6 "gtcaatcatgttcttgtgaatggatttaaaaaaaaggggggggaccgcttggcgcacccaaattcagtgtgggcgagcgcaa"
7 "cggttttggcccttgttagaggcccccgtaaaaaaaagggggggcaattatgagagagctaatctatcgcgtgcgtgttcat"
8 "aacttgagttaaaaaaaagggggggctggggcacatacaagaggagtcttccttatcagttaatgctgtatgacactatgta"
9 "ttggcccattggctaaaagcccaacttgacaaatggaagatagaatccttgcataaaaaaaagggggggaccgaaagggaag"
10 "ctggtgagcaacgacagattcttacgtgcattagctcgcttccggggatctaatagcacgaagcttaaaaaaaaggggggga"
```

We just use the FrequentWords algorithm we made earlier.

In [1]:
def frequency_table(text, k):
    """
    Returns a frequency table of k-mers in a text.

    Args:
        text (str): The text.
        k (int): the length of the k-mers.

    Returns:
        freq_map (dict): The map of each k-mer to its frequency.
    """
    freq_map = {}
    n = len(text)
    # indexing excludes end of range so must add +1 to n-k
    for i in range(0, n-k+1):
        pattern = text[i:i+k]
        if pattern not in freq_map:
            freq_map[pattern] = 1
        else:
            freq_map[pattern] += 1
    return freq_map


def maxmap(input_map):
    """
    Returns the maximum value (not key) in a map.

    Args:
        input_map (dict): the map (type(value) == int)

    Returns:
        max_value (int): the maximum value.
    """
    max_value = max(input_map.values())
    return max_value

    
def frequent_words(text, k):
    """
    Returns the most frequent k-mers from a text.

    Args:
        text (str): the text.
        k (int): the length of the k-mers.

    Returns:
        frequent_patterns (list): the most frequent k-mers.
    """
    frequent_patterns = []
    freq_map = frequency_table(text, k)
    max_num = maxmap(freq_map)
    for pattern, freq in freq_map.items():
        if freq == max_num:
            frequent_patterns.append(pattern)
    return frequent_patterns

In [2]:
text = "atgaccgggatactgataaaaaaaagggggggggcgtacacattagataaacgtatgaagtacgttagactcggcgccgccgacccctattttttgagcagatttagtgacctggaaaaaaaatttgagtacaaaacttttccgaataaaaaaaaagggggggatgagtatccctgggatgacttaaaaaaaagggggggtgctctcccgatttttgaatatgtaggatcattcgccagggtccgagctgagaattggatgaaaaaaaagggggggtccacgcaatcgcgaaccaacgcggacccaaaggcaagaccgataaaggagatcccttttgcggtaatgtgccgggaggctggttacgtagggaagccctaacggacttaataaaaaaaagggggggcttataggtcaatcatgttcttgtgaatggatttaaaaaaaaggggggggaccgcttggcgcacccaaattcagtgtgggcgagcgcaacggttttggcccttgttagaggcccccgtaaaaaaaagggggggcaattatgagagagctaatctatcgcgtgcgtgttcataacttgagttaaaaaaaagggggggctggggcacatacaagaggagtcttccttatcagttaatgctgtatgacactatgtattggcccattggctaaaagcccaacttgacaaatggaagatagaatccttgcataaaaaaaagggggggaccgaaagggaagctggtgagcaacgacagattcttacgtgcattagctcgcttccggggatctaatagcacgaagcttaaaaaaaaggggggga"
frequent_words(text, 15)

['aaaaaaaaggggggg']

But when the regulatory motif has mutations, even the Frequent Words with Mismatches algorithm becomes too slow, and unlike a DnaA box, a regulatory motif only has to appear at least once in each of the regions in the genome associated with the regulatory gene. For this, we might use a brute force algorithm first. 

Given a collection of strings Dna and an integer d, a k-mer is a (k,d)-motif if it appears in every string from Dna with at most d mismatches. For example, the implanted 15-mer in the strings above represents a (15,4)-motif.

**Implanted Motif Problem**: Find all (k, d)-motifs in a collection of strings.

    **Input**: A collection of strings Dna, and integers k and d.
    **Output**: All (k, d)-motifs in Dna.


In [3]:
def hamming(s1, s2):
    """
    Find Hamming distance between two strings
    """
    dist = 0
    for i in range(len(s1)):
        if s1[i] != s2[i]:
            dist += 1
    return dist

    
def neighbors(pattern, d):
    # handle edge cases first
    if d == 0:
        return [pattern] # absolutely necessary to return list
    if len(pattern) == 1:
        return ["A", "T", "G", "C"]
    neighborhood = []
    suffix_neighbors = neighbors(pattern[1:], d)
    for text in suffix_neighbors:
        if hamming(pattern[1:], text) < d:
            for nt in ["A", "T", "G", "C"]:
                neighborhood.append(nt+text)
        else:
            neighborhood.append(pattern[0]+text)
    return neighborhood

In [4]:
def kmer_in_all_dna(kmer, kmers_dict):
    for array in kmers_dict.values():
        if kmer not in array:
            return False
    return True
    
def motif_enumeration(dna, k, d):
    patterns = set()
    kmers_dict = {}
    all_kmers = set()

    for pattern in dna:
        kmers = {pattern[i:i+k] for i in range(len(pattern)-k+1)}
        kmers_dict[pattern] = set()
        for kmer in kmers:
            kmer_neighbors = neighbors(kmer, d)
            for i in kmer_neighbors:
                kmers_dict[pattern].add(i) # d-neighbors of the k-mer
                all_kmers.add(i)
            kmers_dict[pattern].add(kmer) # the k-mer itself
            all_kmers.add(kmer)
                        
    for kmer in all_kmers:
        if kmer_in_all_dna(kmer, kmers_dict):
            patterns.add(kmer)
    
    return patterns

In [5]:
dna = ["AGTGCATCTGATTAGTCGTTGGATG", "GAGTCTCCAGAGTTGCATATGGAAA", "ATCTAAGTATGCACTTCATCAGGCC", "TCCGAAGTTTTGACGCCCGGAATTA", "CACCATTTTGAGTTCTTGCAACCTA", "CAACCTAGGCACCATGGCAAAGTTG", "TAGTAGGGCTTGTATCCCAGAGTGG", "AGTACTTCGCCCTTATCCTACAAGG", "TCCGGGTCTGGTGATCTTCTAGTTC", "AGTTATTGACCTCCAGGCGAATCAC"]
k = 5
d = 2
" ".join(motif_enumeration(dna, k, d))

poop = ["brown", "brown", "slushy", "hard"]
set(poop)

{'brown', 'hard', 'slushy'}

In [6]:
dna = ["TCGGGGGTTTTT",
    "CCGGTGACTTAC",
    "ACGGGGATTTTC",
    "TTGGGGACTTTT",
    "AAGGGGACTTCC",
    "TTGGGGACTTCC",
    "TCGGGGATTCAT",
    "TCGGGGATTCCT",
    "TAGGGGAACTAC",
    "TCGGGTATAACC"]

In [7]:
def score(dna):
    score_dicts = []
    scores = []
    for i in range(len(dna[0])):
        nt_counts = {"A":0, "T":0, "G":0, "C":0}
        for seq in dna:
            nt_counts[seq[i]] += 1
        score_dicts.append(nt_counts)
    for d in score_dicts:
        print(d)
        dict_score = 0
        for key, value in d.items():
            if key != max(d, key=d.get):
                dict_score += value
        scores.append(dict_score)
    return scores

In [8]:
score(dna)

{'A': 2, 'T': 7, 'G': 0, 'C': 1}
{'A': 2, 'T': 2, 'G': 0, 'C': 6}
{'A': 0, 'T': 0, 'G': 10, 'C': 0}
{'A': 0, 'T': 0, 'G': 10, 'C': 0}
{'A': 0, 'T': 1, 'G': 9, 'C': 0}
{'A': 0, 'T': 1, 'G': 9, 'C': 0}
{'A': 9, 'T': 0, 'G': 1, 'C': 0}
{'A': 1, 'T': 5, 'G': 0, 'C': 4}
{'A': 1, 'T': 8, 'G': 0, 'C': 1}
{'A': 1, 'T': 7, 'G': 0, 'C': 2}
{'A': 3, 'T': 3, 'G': 0, 'C': 4}
{'A': 0, 'T': 4, 'G': 0, 'C': 6}


[3, 4, 0, 0, 1, 1, 1, 5, 2, 3, 6, 4]

In [9]:
def count(dna):
    score_dicts = []
    counts = {}
    for i in range(len(dna[0])):
        nt_counts = {"A":0, "T":0, "G":0, "C":0}
        for seq in dna:
            nt_counts[seq[i]] += 1
        for k, v in nt_counts.items(): # necessary because of floating point error if done in prev for in
            nt_counts[k] = v/len(dna)
        score_dicts.append(nt_counts)
    return(score_dicts)

In [10]:
count(dna)

[{'A': 0.2, 'T': 0.7, 'G': 0.0, 'C': 0.1},
 {'A': 0.2, 'T': 0.2, 'G': 0.0, 'C': 0.6},
 {'A': 0.0, 'T': 0.0, 'G': 1.0, 'C': 0.0},
 {'A': 0.0, 'T': 0.0, 'G': 1.0, 'C': 0.0},
 {'A': 0.0, 'T': 0.1, 'G': 0.9, 'C': 0.0},
 {'A': 0.0, 'T': 0.1, 'G': 0.9, 'C': 0.0},
 {'A': 0.9, 'T': 0.0, 'G': 0.1, 'C': 0.0},
 {'A': 0.1, 'T': 0.5, 'G': 0.0, 'C': 0.4},
 {'A': 0.1, 'T': 0.8, 'G': 0.0, 'C': 0.1},
 {'A': 0.1, 'T': 0.7, 'G': 0.0, 'C': 0.2},
 {'A': 0.3, 'T': 0.3, 'G': 0.0, 'C': 0.4},
 {'A': 0.0, 'T': 0.4, 'G': 0.0, 'C': 0.6}]

In [11]:
import math
def safe_log2(value):
    try:
        return math.log2(value)
    except:
        return 0
        
def entropy(d):
    """
    Calculate entropy of given a dictionary with probabilities for each nucleotide
    """
    plogp = [value*safe_log2(value) for key, value in d.items()]
    ent = -1*sum(plogp)
    return ent

def total_entropy(dna):
    count_matrix = count(dna)
    entropies = [entropy(d) for d in count_matrix]
    return sum(entropies)
        
        
        

In [12]:
entropy({'A': 0.2, 'T': 0.2, 'G': 0.0, 'C': 0.6})

1.3709505944546687

In [13]:
total_entropy(dna)

9.916290005356972

In [14]:
# stolen code from comment section
def hamming_dist(p,q):
    dist = 0
    if(len(p) != len(q)):
        return "Unable to compute."
    else:
        for i in range(0,len(p)):
            if p[i] == q[i]:
                next
            else:
                dist+=1
    return dist
    
def closest_match(dna,pattern):
    """
    given a dna fragment and a pattern with length=k, 
    find a substring k-mer in dna fragment that has the 
    smallest hamming distance from the pattern.
    """
    distances = {}
    for i in range(0,len(dna)-len(pattern)+1):
        kmer = dna[i:i+len(pattern)]
        kmer_dist = hamming_dist(kmer,pattern)
        distances[kmer] = kmer_dist
    min_val = min(distances, key=distances.get)
    return min_val

def median_string(dnas,pattern):
    """
    Within each dna in dnas, find a k-mer min_val such that it is the closest match to the given pattern. 
    If there is a substring "AAA", it will be the closest match.
    """
    kmers = []
    for d in dnas:
        min_val = closest_match(d,pattern)
        kmers.append(min_val)
    return kmers

print(*median_string(["TTACCTTAAC","GATATCTGTC","ACGGCGTTCG","CCCTAAAGAG","CGTCAGAGGT"],"AAA"))


TAA ATA ACG AAA AGA


```
DistanceBetweenPatternAndStrings(Pattern, Dna)
    k ← |Pattern|
    distance ← 0
    for each string Text in Dna
        HammingDistance ← ∞
        for each k-mer Pattern’ in Text
            if HammingDistance > HammingDistance(Pattern, Pattern’)
                HammingDistance ← HammingDistance(Pattern, Pattern’)
        distance ← distance + HammingDistance
    return distance
```

In [15]:
# my own implementation
def DistanceBetweenPatternAndStrings(pattern, dnas):
    k = len(pattern)
    distance = 0
    for dna in dnas:
        hamming_dist = len(dna)
        for i in range(len(dna)-len(pattern)+1):
            kmer = dna[i:i+len(pattern)]
            current_hamming = hamming(pattern, kmer)
            if hamming_dist > current_hamming:
                hamming_dist = current_hamming
        distance += hamming_dist
    return distance

In [16]:
pattern = "TTACG"
dnas = ["GCCCGTACGCGCCGGGCATTAATTGAGGGGCCAAAACCTTGACGAGGGAAAAGTGCCTACATAACTAGCCGTAACATCTGTCCGACTGCGAAACAAGGGAAGAGC",
"AAAGTACGTTATATATGCTCCCGCCACATACGGCGGGCTAATTACAATGCAAATGCCGCTTAATGATTGCAATGGTGATAACAGTATGGCAATGTTTAAGGGACG",
"GCGCGATACCACCGTGAGGCGGAGATGAGTGGCGTTCGTCCGTAACTAGTGAGCATGTTACCACGGGTCCAATAGATCCCATAGGACCCTCAATTATGGTGGAGA",
"CTTCTGTGCACGGATATTTTACCCGGGAGAGATGCGGCTCTGGTTGCGACAATAACCGGGGTGGGTTTAAGTTGGAACTGGTGACGATAGTTAATAGACCATCGT", 
"TCAAATAGCTCCTGGATCAGGATTGATTAGCGTATGACGGGTCACCGCGCTCATGCATTCAGCGTATAGAAGTCCCCTCCGACGCAAACCCGCGCCGAGACGCTC",
"AATCGAATGGGGGCGGTTTTGCACCAGTTCGATAGGTAGTGCCCCGAATACACACGTCGTGCTCATAACATTCGCTGCCTTTTGGGCTGGTAATTTTGGAAATCT",
"CGGGAGGTATCCACTGAATCCGACCAGCCCTGGGTTTTGGTTCACCCGCTTCTCGTCTTGGAGTACACGTTTGCTGCGCGCGATTGTCATTTTGGATATGTCAAC",
"CTGAGACCAATCGGCGCATATTAGTTGACCGATGTACCAAACTGATAACACACCCAGCTTCCTTCATCCAATCATCGGCTAGTGAGCTAGAGCCGGCCCGGCCCT",
"CAAGCAGCATCCCCTCAAGAGCCCATTTTAAAGCGCGAGCCATTTAGGGCCGTTTTTCCTGTTTCCTCTCTCGTAAGCAGGTTGCTCCTACCTACAACAGACCAA",
"GTAGAAGTGATGACTGTCGGCTGTTGTTGTGACTGGTCTCAACCGTGTGGCGTCAAGATAGTGCGGTATCCTCTTCGATTATCAGATTTTCTGGGAGAGATTTAC",
"CGCCACATGATGAATATAACATCTGTAGGGGTGTGTCGACAATGGCCTTGATGTCTCTTTCCCCGACGTCTGTGAAACAAAATTTTGTCGGCAGGTGGGCCCACT",
"GACAAGTATTCACGCTCGTTTATTTGGGTTGAAGATTTAGCCTATCGACAACAGCCGTGGGGGGCGGTGGGCGGCAGAACTAACTCTATACTCGGATACCTTATA",
"AAGGACACTGGTACGATCCTAGAAGGGGCAATGAGAATATGGACACTGGTCTAGAATGGATCCGGTAAGCTCCTCATAAGCCACTATCGGGATGGCCGGATTCTC",
"GCTGCAGGCCGCTTTTCTCCGTTTAGGCCCCACCTGACGGGGTAACGAGTTTAGCGAGCCCTCTTTGAAAGGGGGAACGCATTTGATGGTTGCAGTAGACATAGT",
"CAGATTTACATAACATCTAATTATATAGGCACTACCTACATGTGGTGATGTGCACGTGAACACGGAGTGGGCGCATTCCGCGTTATGTATGCCAGAGACCTTTCA",
"CAATGGGTCTTTTATAACAAGGTTATACAATATTAGAGAATACCAAACGACGGCCCGGTATACCTGAGTTGCTCCAAACCGACTGATTCTGGTTGAACAAAGGTT",
"TACTACCATGAGTTTATCGTCAGAGTATTCGCACTCGTAGACGAGATTCTGAAGCGCATTAACTGCTCCTTGGGCTGGGTTCCAAGGTCGGTTAGTCTTAGGGAT",
"TGTAAGCGCCGCCTACAGCGTTACTCCTCTACATCAATCATAGTTGGTAGAAAGTCAGTAGGTACCCAACGACACAATGTTTATAGATCCTAATGTGAGTGGCTC",
"CCTTACGTTGAGGTATGCTTACTTCGGCAAATGAACCGTTCGAACAAACCGGGTCAAATAGCTAGAGAGACTCAGAGTGGAGGCCATAATATATGGAGAGACATG",
"GGGTAATTACTCTAATATTCTCAAATCACGGCAATACGGAGATAAACCTCTTTACCGACAGAATCTTAGGAAGCACATAATAACTAGGTAGGTAACATTATCAAC",
"CCCATTCAGGTACCATGAAAAAAATTTATTTTCATGGTTGCCAAAATCCACGTTACACATGCCGGCAGCTTGTGGGCAACTGAGCAAGTCCGAGGACGTGAAGCT",
"TTCATTACTGTGCATCAGGCACTGTACGTCTACCCAACTTAAGCTTATGATAAACGCTTCAGACCTTGGGTATTACCTCTGCTTGCTGACCTGGATAGCGCAAAT",
"TAGTGCAGCTTGCAAGTAGAAAACTATATTCGGGGTCCGCTCACTATGAGGGTATACGATCCTACTGGGAGGCCTCTATGAAACACACCGTTAGGGGAGCCCATA",
"CAGCCCATACATAGAAGAGGACGCACTCTACGTGTTGCATCTAGTAGTCAGCACTAATTACAACTCTATCCATAGTAGCAAAGGAATAAACGACAAAAGGTCTAA",
"GTATGCTTCGTCTGTCTCACATAAACAGCGGATACTGTAAGTAACAAACAGATTGACCATAGCGTCAGGTCCGCTAGCACCCCCTGGCATTGCGGGACGAGTCTC",
"CAGTCTTTAACCAGGACCAACAATCCTCGTAAACCTGTGGCTCTCTCCAGGTCTGTTTATTTATTGCATCAGCGTCTCGCACCTTGTGCACAAACGACGTCAATG",
"GCAATGTAGATTTAACATTGTCGACAGTTACCCAGAGCCTGAGACACCTTTCGTCTTTCTAGGCCACGACTTCGCCAGGTTAGATAAGTGAGCCCGTGTGGAAGG",
"CGCATAACAGACAAATGTAACTGTGTACGTCTAAGGCGCCCTTTTTGTAGTCATCCCACCGCCACTATGGCTTGCTCCGTTCTAGCTGTGGTAGACGGCGCCCCA",
"CACCGAGCATGAATTATCAGTTCTTTTGACTGGGAGGACTGATTGGTGGATAACCTGAACGGTGGATTACGGGTAACTCACTTAGCATGACCCCACTAGGTTATA",
"CAGGTGTCCGTTTGTCAATACTCTCCAGAATGTAGGAAGTTCGTACTTACTCAGGCTCTTCCGGTTGCCTCAACCCCTCTAAATTGTCACGACGTTTGTCGAGTA",
"AGTAACCCGAAAGCCTATATGTTGAATTCCTGGTGTTGCAAGTATATGCCAGAGCAGATTTTGGAAGATCCCGCCAGGCGCGGCTTGTTAAACTATTTACTATTC",
"CGATCGGGGTAAAGCAGCTTCAAGCCCTCCGTTAAGGCGCAGTCTACTGCATCTTCGTCAGTAATCTTAAGGCAGCGCATCGTTTGAGTACTCGTGGATTAACGG",
"CCTGAAGCCCGACGAGAAATGGGAACAACTTTTAGCCGTTCGAACGTTCTGCATCGGCGGAAGCTTGATACGTATCTTGCGCATTTGGCGACCAGACAAGCTAAG",
"TTGGCAACGAACCTGCAGATTTCTAACGGAACGTTCTTCCGCGCCATTGAAGTATGACTCGCCTACCAGTTTCGGAATGTCGTAGCATTACCTACTTTACTGGTC"]
        

DistanceBetweenPatternAndStrings(pattern, dnas)

39

```
MedianString(Dna, k)
    distance ← ∞
    Patterns ← AllStrings(k)
    for i ← 0 to |Patterns|
        Pattern ← Patterns[i]
        if distance > DistanceBetweenPatternAndStrings(Pattern, Dna)
            distance ← DistanceBetweenPatternAndStrings(Pattern, Dna)
            Median ← Pattern
    return Median
```

In [17]:
def medianString(dnas, k):
    distance = float('inf')
    patterns = neighbors("".join(["A" for i in range(k)]), k)
    for pattern in patterns:
        current_distance = DistanceBetweenPatternAndStrings(pattern, dnas)
        if distance > current_distance:
            distance = current_distance
            median = pattern
    return median

In [36]:
k=6

dnas = [
    "TATGCCATCGGCCCAAGAGCAGGTGTGCGGCTGAATGAGTCA",
    "GATAGTTGTTAATGACTACTCTTTATCGGGACCCGTCATCCG",
    "CTGGGCCGTAGTAAACGTGGCGCCATCGGTATAGCACGTAAT",
    "CGTACCGCCTGCATCTTTATCGGAACTCCGATCCAACCCATC",
    "ATCGGTGATCAGCTTTGCCCTCCCGGCTGGGAGTACCTACAA",
    "AGCAAATTAGGCGATGGCACGACGCTCAGCCCTACGATCGGC",
    "GGCGGATCGTCCGGTCGGAGCCTGATCGGCCTTGTTTCACAC",
    "AGTGCCTAACCTGTCGTGATCGGAAAACCGAACATGGCAGGT",
    "TTGAGTCCGGAAAATCCACCAGGAAATATTAATGACATCGGT",
    "TCAAGGTCTCATTCGCTAGACCCAATCGGCCGACATTCGCGG"
]
medianString(dnas, k)

'ATCGGC'

In [37]:
from itertools import product

def hamming_distance(pattern1, pattern2):
    """Calculate the Hamming distance between two strings."""
    return sum(c1 != c2 for c1, c2 in zip(pattern1, pattern2))

def median_string(dna, k):
    """Find the median string with the smallest total Hamming distance to all strings in the given DNA sequences."""
    distance = float('inf')
    median = None
    
    # Generate all possible k-mers
    for pattern in [''.join(p) for p in product('ACGT', repeat=k)]:
        # Calculate the total distance between the current pattern and all strings in DNA
        total_distance = sum(min(hamming_distance(pattern, dna_str[j:j+k]) for j in range(len(dna_str) - k + 1)) for dna_str in dna)
        
        # Update the median and distance if the total distance is smaller
        if total_distance < distance:
            distance = total_distance
            median = pattern
    
    return median

In [38]:
median_string(dnas, k)

'ATCGGC'